In [1]:
from bs4 import BeautifulSoup as bs
import lxml.html as lh
from lxml.html import builder as E
from bs4 import BeautifulSoup
import requests
import json
import re
import csv
import pandas as pd

In [2]:
df=pd.read_csv("../extracted_data/data_1000_plus_1.csv")
df.head()

,USDOT,Company Name,Operating status,DBA,Entity type,Operations,Classification,Docket #,DUNS #,SCAC,...,Email,Web,Mailing address,Representative,Power units,Autonomous units,Electric units,Drivers,SmartWay,CARB TRU
0,382472,FLORAL MARKETING SOLUTIONS INC,NONE,NaN,NaN,NaN,NaN,MC313239,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not a,Not 100% compliant
1,390424,BLUE BIRD PRODUCTS INC,ACTIVE,NaN,Carrier,Intrastate Non-Hazmat,Private Property,NaN,16834004.0,NaN,...,NaN,NaN,"15135 HAMILTON AVE HIGHLAND PARK, MI 48203-3725",NaN,22.0,NaN,NaN,NaN,Not a,Not 100% compliant
2,582596,TRANSPORTATION COMMODITIES INC,AUTHORIZED FOR PROPERTY,TCI LEASING AND RENTALS,Carrier,Interstate,Authorized For Hire LEASING AN,MC252726,NaN,NaN,...,vollhardtt@tci-leasing.com,NaN,"4950 TRIGGS ST COMMERCE, CA 90022",ANDREW FLYNN,1003.0,NaN,NaN,14,Not a,Not 100% compliant
3,352900,WALTER R ANSARDI,ACTIVE,NaN,Carrier,Intrastate Non-Hazmat,APPLYING F,NaN,NaN,NaN,...,NaN,NaN,"69296 ARCHIE SINGLETARY RD PEARL RIVER, LA 70452",NaN,3.0,NaN,NaN,2,Not a,Not 100% compliant
4,328986,CEMEX CONSTRUCTION MATERIALS FLORIDA LLC,ACTIVE,CEMEX,Carrier,Interstate,Private Property,NaN,557093945.0,NaN,...,briank.currie@cemex.com,NaN,"1501 BELVEDERE ROAD WEST PALM BEACH, FL 33406",RYAN MAHONEY DONALD LAFAVERS,1004.0,NaN,NaN,879,Not a,Not 100% compliant


In [3]:
with open ("usdot_1000+.txt", "r") as myfile:
    data = myfile.read()
data=data.replace("[","").replace("]","").replace("'","").replace('"',"")
_all_usdots = data.split(',')
all_usdots=[]
for element in _all_usdots:
    all_usdots.append(element.strip())

In [4]:
extracted_usdots=df["USDOT"].astype(str).to_list()

In [5]:
def common_member(a, b):
    result = [i for i in a if i in b]
    return result
common=common_member(all_usdots,extracted_usdots)

In [6]:
for element in common:
    all_usdots.remove(element)
remain=all_usdots

In [7]:
len(all_usdots)

670

extracting

In [8]:
def get_company_page(usdot_no):
    cookies = {
        'X-KEY': 'eyJpdiI6InVVc01BR3dvYm8xNmEyYko1NnYwTGc9PSIsInZhbHVlIjoiVXBpZDlCbThzdkNDWkcyVityU2UzMXUwaEpOV1o3MG13T0o3Q3o3U0xkdWFnVkdnSll0eHRaZGltV1QwRk1vdUhKVVBTbkkvcEZuV3IzN2VjSXA5OFBENVNCNUlIcHBFQThVbHd5TityRkM3ZURmUWpMWWF5RzU2R0cvTXdFd2M5bUtacVpWUVJBMTNISEM0OGN5bkhvczJDNnlucUJvYWpDaHV2ZS9FWnRZZExXdExHVkFWajFCZDRGU0JMeXl5SG56RmE2bVdtYlpMMFVrWnUydExXa2xRNVVpT1ZpUHhLT3VneXFtbFE4bFl2em1IOVpONlZySExnL0xJaThSVyIsIm1hYyI6IjI1NDI1YWEzZmUwMzRhNDdjODQ3MDhhNDA1YjZiMDBhYzJhMzRiYjU5OTk4YmUxN2NhMzg1OWU0ZDg0NTQ2ZDQifQ%3D%3D',
        '_ga': 'GA1.1.1581905416.1655785741',
        '__gads': 'ID=99f07c8c1a291d4d-2257f1a267d40030:T=1655785920:RT=1655785920:S=ALNI_MYlSRLddQtXSEyb4lHHQsOjsdUeUg',
        '__gpi': 'UID=000006d14ebceca5:T=1655785920:RT=1655807753:S=ALNI_MZh0MOvwE_5nwCwuuuhUqb7KFLpkQ',
        '_ga_2CX7JCZ1RS': 'GS1.1.1655807743.7.1.1655807957.0',
    }

    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8',
        'Cache-Control': 'no-cache',
        'Connection': 'keep-alive',
        # Requests sorts cookies= alphabetically
        # 'Cookie': 'X-KEY=eyJpdiI6InVVc01BR3dvYm8xNmEyYko1NnYwTGc9PSIsInZhbHVlIjoiVXBpZDlCbThzdkNDWkcyVityU2UzMXUwaEpOV1o3MG13T0o3Q3o3U0xkdWFnVkdnSll0eHRaZGltV1QwRk1vdUhKVVBTbkkvcEZuV3IzN2VjSXA5OFBENVNCNUlIcHBFQThVbHd5TityRkM3ZURmUWpMWWF5RzU2R0cvTXdFd2M5bUtacVpWUVJBMTNISEM0OGN5bkhvczJDNnlucUJvYWpDaHV2ZS9FWnRZZExXdExHVkFWajFCZDRGU0JMeXl5SG56RmE2bVdtYlpMMFVrWnUydExXa2xRNVVpT1ZpUHhLT3VneXFtbFE4bFl2em1IOVpONlZySExnL0xJaThSVyIsIm1hYyI6IjI1NDI1YWEzZmUwMzRhNDdjODQ3MDhhNDA1YjZiMDBhYzJhMzRiYjU5OTk4YmUxN2NhMzg1OWU0ZDg0NTQ2ZDQifQ%3D%3D; _ga=GA1.1.1581905416.1655785741; __gads=ID=99f07c8c1a291d4d-2257f1a267d40030:T=1655785920:RT=1655785920:S=ALNI_MYlSRLddQtXSEyb4lHHQsOjsdUeUg; __gpi=UID=000006d14ebceca5:T=1655785920:RT=1655807753:S=ALNI_MZh0MOvwE_5nwCwuuuhUqb7KFLpkQ; _ga_2CX7JCZ1RS=GS1.1.1655807743.7.1.1655807957.0',
        'Pragma': 'no-cache',
        'Referer': 'https://usdot411.com/user/search/advanced',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-User': '?1',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    return requests.get('https://usdot411.com/'+str(usdot_no), cookies=cookies, headers=headers)

In [9]:
company_page=get_company_page(all_usdots[2]).text
soup = bs(company_page)               
prettyHTML = soup.prettify()
cleantext = BeautifulSoup(prettyHTML, "lxml").text.strip('\n')
cleantext=' '.join(cleantext.split())

In [10]:
cleantext

'USDOT 377707 US DOT 411 Search Monitoring Analytics USDOT 377707 No data available There is no information available for USDOT 377707 usdot411.com © 2020 About Contacts Privacy Terms Justin Lu Premium Trial Search Monitoring Analytics Account Directory About Contacts Privacy Policy Terms of Use Sign out'

In [11]:
company_info=["Operating status","DBA","Entity type","Operations","Classification","Docket #","DUNS #","SCAC","MCS-150 form date","MCS-150 Mileage/Year","Fleet Details"]
contact_info=["Business address","Phone","Fax","Email","Web","Mailing address","Representative","Commodities"]
fleet_info=["Power units", "Autonomous units", "Electric units", "Drivers", "SmartWay", "CARB TRU","Contacts"]
all_records=[]

In [12]:
def extract(usdot_no):
    data={}
    company_page=get_company_page(usdot_no).text
    soup = bs(company_page)               
    prettyHTML = soup.prettify()
    cleantext = BeautifulSoup(prettyHTML, "lxml").text.strip('\n')
    cleantext=' '.join(cleantext.split())

    #usdot number
    data["USDOT"]=number

    #Company Name
    try:
        data["Company Name"]=cleantext.split("USDOT")[0].replace("(","").strip()
    except:
        data["Company Name"]=None
    try:
        company_content=cleantext.split("Company")[1]
        contact_content=cleantext.split("Contacts")[1].split("Building Materials")[0]

        #company details
        for i in range(len(company_info)):
            if company_info[i]=="Fleet Details":
                break
            try:
                data[company_info[i]]=company_content.split(company_info[i+1])[0].split(company_info[i])[1].strip()
            except:
                data[company_info[i]]=None
        
        #contact details
        for i in range(len(contact_info)):
            if contact_info[i]=="Commodities":
                break
            try:
                data[contact_info[i]]=contact_content.split(contact_info[i+1])[0].split(contact_info[i])[1].strip()
            except:
                data[contact_info[i]]=None
        
        #fleet details
        for i in range(len(fleet_info)):
            if fleet_info[i]=="Contacts":
                break
            try:
                data[fleet_info[i]]=company_content.split(fleet_info[i+1])[0].split(fleet_info[i])[1].strip()
            except:
                data[fleet_info[i]]=None
    except:
        data["data"]=cleantext
    return data

In [13]:
def appendtext(fname,data):
	with open(fname,'a+') as f:
		f.write(data)
		f.write("\n")
	f.close()

In [14]:
for number in all_usdots:
    try:
        output=extract(number)
        all_records.append(output)
        appendtext("data.txt",str(output))
    # print("\n")
    # print(output)
    except:
        pass

In [15]:
all_records[:2]

[{'USDOT': '339643',
  'Company Name': '',
  'data': 'USDOT 339643 US DOT 411 Search Monitoring Analytics USDOT 339643 No data available There is no information available for USDOT 339643 usdot411.com © 2020 About Contacts Privacy Terms Justin Lu Premium Trial Search Monitoring Analytics Account Directory About Contacts Privacy Policy Terms of Use Sign out'},
 {'USDOT': '359166',
  'Company Name': '',
  'data': 'USDOT 359166 US DOT 411 Search Monitoring Analytics USDOT 359166 No data available There is no information available for USDOT 359166 usdot411.com © 2020 About Contacts Privacy Terms Justin Lu Premium Trial Search Monitoring Analytics Account Directory About Contacts Privacy Policy Terms of Use Sign out'}]